In [ ]:
# Vitug, Angelo G.
# Date of Creation: 11/15/24
# Date of Submission: 11/18/24

In [ ]:
# Problem: Employee Monitoring System
import pandas as pd
from tkinter import *
from tkinter.scrolledtext import *
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
pd.set_option('future.no_silent_downcasting', True)

class EMSystem:

    def __init__(self):
        self.__currEmployeeList = ""
        self.__logTime = ""
        self.__employeeLogDF = pd.DataFrame(columns = ['ID', 'Employee', 'Status','Hours', 'Added Hrs', 'Total Hrs'])
        self.__logNum = 0

    def setEmployeeList(self, filename):
        try:
            if filename[-4:] == '.csv':
                pd.read_csv(filename)
                self.__currEmployeeList = filename
                print(f"{filename} set as Employee List.")
            elif filename[-4:] == '.txt':
                pd.read_csv(filename, delimiter = '\t')
                self.__currEmployeeList = filename
                print(f"{filename} set as Employee List.")
            else:
                print("Invalid file type. Only .csv or .txt are accepted. Please try again.")
        except FileNotFoundError:
            print("File could not be found. Please try again.")
        except:
            print("The input was invalid. Please try again.")

    def printEmployeeList(self):
        try:
            if self.__currEmployeeList[-4:] == '.txt':
                text = pd.read_csv(self.__currEmployeeList, delimiter = '\t')
            elif self.__currEmployeeList[-4:] == '.csv':
                text = pd.read_csv(self.__currEmployeeList)
            print(f"{text}")
            return text
        except FileNotFoundError:
            print("File could not be found. Please try again.")
        except:
            print("An employee list is not set. Please set one and try again.")

    def makeNewLog(self,time):
        print(f"Timestamp log: {time}")
        self.__logTime = time
        self.__employeeLogDF = self.__employeeLogDF.iloc[0:0]

    def addEmployeeToLog(self, employeeID, status, timeIn):
        if self.__currEmployeeList[-4:] == '.txt':
            inputEmployeeListDF = pd.DataFrame(pd.read_csv(self.__currEmployeeList, delimiter = '\t'))
        elif self.__currEmployeeList[-4:] == '.csv':
            inputEmployeeListDF = pd.DataFrame(pd.read_csv(self.__currEmployeeList))

        appendingRow = inputEmployeeListDF.loc[inputEmployeeListDF['ID'] == employeeID]
        self.__employeeLogDF = pd.concat([appendingRow, self.__employeeLogDF], ignore_index = True)
        self.__employeeLogDF = self.__employeeLogDF.fillna(value = 0)

        employeeInitialHours = self.__employeeLogDF.loc[self.__employeeLogDF['ID'] == employeeID, "Hours"]

        if status == 'out':
            logOutHour = int(timeIn[:2])
            logOutMins = int(timeIn[3:])
            logTimeHour = int(self.__logTime[:2])
            logTimeMins = int(self.__logTime[3:])

            addHours = (((logTimeHour * 60) + logTimeMins) - ((logOutHour * 60) + logOutMins)) // 60

            self.__employeeLogDF.loc[self.__employeeLogDF['ID'] == employeeID, "Added Hrs"] = addHours
            self.__employeeLogDF.loc[self.__employeeLogDF['ID'] == employeeID, "Total Hrs"] = addHours + employeeInitialHours
            self.__employeeLogDF.loc[self.__employeeLogDF['ID'] == employeeID, "Status"] = "Out"
            print("Employee status was successfully set to OUT. Appropriate time has been added to their hours.")
        elif status == 'in':
            self.__employeeLogDF.loc[self.__employeeLogDF['ID'] == employeeID, "Added Hrs"] = 0
            self.__employeeLogDF.loc[self.__employeeLogDF['ID'] == employeeID, "Total Hrs"] = employeeInitialHours
            self.__employeeLogDF.loc[self.__employeeLogDF['ID'] == employeeID, "Status"] = "In"
            print("Employee was added to the log list and their status was set to IN.")

        print("Updated employee log:")
        print(f"{self.__employeeLogDF}")

    def getUpdatedLog(self):
        return self.__employeeLogDF

    def createTimestampLog(self):
        self.__logNum += 1
        print("Creating timestamp log")
        print(f"Timestamp Log {self.__logTime}")
        print(f"{self.__employeeLogDF}")

        self.__employeeLogDF.to_csv(f"log{self.__logNum}.txt", sep = '\t', mode = 'w', header = True, index = False)

        originalCopy = open(f"log{self.__logNum}.txt", 'r')
        originalText = originalCopy.readlines()
        originalCopy.close()
        originalCopy = open(f"log{self.__logNum}.txt", 'w')
        originalCopy.write(f"Timestamp Log {self.__logTime}\n")
        [originalCopy.write(x) for x in originalText]
        originalCopy.close()
    
    def createLogSummaryReport(self):
        try:
            text = pd.read_csv(f"log{self.__logNum}.txt", delimiter = '\t')
            return text
        except FileNotFoundError:
            print("File could not be found. Please try again.")
        except:
            print("The input was invalid. Please try again.")

In [ ]:
window = Tk()
window.title("Mindful EM System")
window.resizable(False, False)

ems = EMSystem()

fileLoadLabel = Label(window, text="Type the employee list file in the text box or\nOpen a file by pressing the 'Open file' button (.csv and .txt only)")
fileLoadLabel.grid(row=0, column=0)

fileEntry = Entry(window, width=50)
fileEntry.grid(row=1, column=0, padx=10)
statusEntryLabel = Label(window, text="")
statusEntryLabel.grid(row=2, column=0)

def fileSetList():
    filename = fileEntry.get()
    try:
        if filename[-4:] == '.csv' or filename[-4:] == '.txt':
            ems.setEmployeeList(filename)
            statusEntryLabel.configure(text=f"{filename} set as Employee List")
        else:
            statusEntryLabel.configure(text="File could not be read. Please try again.")
    except:
        statusEntryLabel.configure(text="Something went wrong. Please try again.")

setFileButton = Button(text="Set File", command=fileSetList)
setFileButton.grid(row=3, column=0)

employeeScrolledText = ScrolledText(window, state=DISABLED)
employeeScrolledText.grid(row=0, column=1, rowspan=14, pady=10)

def printEmployeeList():
    try:
        employeeScrolledText.configure(state=NORMAL)
        employeeScrolledText.delete('1.0', END)
        employeeScrolledText.insert(INSERT, ems.printEmployeeList())
        employeeScrolledText.configure(state=DISABLED)
        statusEntryLabel.configure(text="")
    except:
        statusEntryLabel.configure(text="Something went wrong. Please try again.")

openFileButton = Button(text="Load Employee List", command=printEmployeeList)
openFileButton.grid(row=6, column=0)

def logSummaryReport():
    try:
        employeeScrolledText.configure(state=NORMAL)
        employeeScrolledText.delete('1.0', END)
        employeeScrolledText.insert(INSERT, ems.createLogSummaryReport())
        employeeScrolledText.configure(state=DISABLED)
        statusEntryLabel.configure(text="")
    except:
        statusEntryLabel.configure(text="Something went wrong. Please try again.")

logSummaryButton = Button(text="Load Last Log", command=logSummaryReport)
logSummaryButton.grid(row=7, column=0)

def selectFile():
    try:
        filetypes = (
            ('Text Files', '*.txt'),
            ('Comma-Separated Values', '*.csv')
        )

        filename = fd.askopenfilename(
            title = 'Open a file',
            initialdir = '/FINALProject-VITUGGelo-236727',
            filetypes=filetypes
        )

        if filename != "":
            ems.setEmployeeList(filename)
            statusEntryLabel.configure(text="An employee list was set through file selection.")
            showinfo(title = 'File opened', message = "A file was successfully set as the employee list!")
        else:
            showinfo(title = 'No file selected', message = "A file was not selected.")
    except:
        showinfo(title = 'Oh no!', message = "Something went wrong. Please try again.")

openFileButton = Button(text="Open file", command=selectFile)
openFileButton.grid(row=4, column=0)

timestampLogTimeLabel = Label(window, text="Start a timestamp log here.\nType the time in military time, incl. the colon. (e.g. 00:00)")
timestampLogTimeLabel.grid(row=0, column=2)

timestampLogTimeEntry = Entry(window, width=50)
timestampLogTimeEntry.grid(row=1, column=2, padx=10)

timestampLogStatusLabel = Label(window, text="")
timestampLogStatusLabel.grid(row=13, column=2, pady=10)

def startTimestampLog():
    time = timestampLogTimeEntry.get()
    try:
        if time[2] == ':' and len(time) == 5:
            global timestampHours
            timestampHours = int(time[:2])
            minutes = int(time[3:])
            if timestampHours > -1 and timestampHours < 24 and minutes > -1 and minutes < 60:
                ems.makeNewLog(time)
                timestampLogAddButton.configure(state=NORMAL)
                generateLogFileButton.configure(state=NORMAL)
                timestampLogIDLabel.configure(text="Log started,\nInput an employee ID")
                timestampLogStatusLabel.configure(text="")
                printEmployeeList()
            else:
                timestampLogStatusLabel.configure(text="Time input was invalid. Please try again.")
        else:
            timestampLogStatusLabel.configure(text="Time input was invalid. Please try again.")
    except:
        timestampLogStatusLabel.configure(text="Something went wrong. Please try again.")

timestampLogTimeButton = Button(window, text="Start a log", command=startTimestampLog)
timestampLogTimeButton.grid(row=2, column=2)

timestampLogIDLabel = Label(window, text="Once a log has been started,\nInput an employee ID")
timestampLogIDLabel.grid(row=3, column=2)

timestampLogIDEntry = Entry(window, width=50)
timestampLogIDEntry.grid(row=4, column=2)

employeeStatusLabel = Label(window, text="Input employee status (IN/OUT)")
employeeStatusLabel.grid(row=6, column=2)
employeeStatusEntry = Entry(window, width=25)
employeeStatusEntry.grid(row=7, column=2)

timeInLabel = Label(window, text="Input time for time-in if employee status is OUT")
timeInLabel.grid(row=8, column=2)
timeInEntry = Entry(window, width=25)
timeInEntry.grid(row=9, column=2)

def addEmployeeToLog():
    try:
        employeeID = 0
        catchInput = str(timestampLogIDEntry.get()).strip()
        if catchInput.isnumeric():
            employeeID = int(catchInput)
            employeeStatus = str(employeeStatusEntry.get()).lower().strip()
            employeeTimeIn = str(timeInEntry.get())
            
            currentLog = ems.getUpdatedLog()

            if currentLog['ID'].isin([employeeID]).any():
                timestampLogStatusLabel.configure(text="Employee is already in the log. Please try again.")
            else:
                if employeeStatus == 'out':
                    if employeeTimeIn[2] == ':' and len(employeeTimeIn) == 5:
                        hours = int(employeeTimeIn[:2])
                        minutes = int(employeeTimeIn[3:])
                        if hours > -1 and hours < 24 and minutes > -1 and minutes < 60 and (timestampHours-hours) > -1:
                            ems.addEmployeeToLog(employeeID, employeeStatus, employeeTimeIn)
                            employeeScrolledText.configure(state=NORMAL)
                            employeeScrolledText.delete('1.0', END)
                            employeeScrolledText.insert(INSERT, ems.getUpdatedLog())
                            employeeScrolledText.configure(state=DISABLED)
                            timestampLogStatusLabel.configure(text="")
                        else:
                            timestampLogStatusLabel.configure(text="Time in input was invalid. Please try again.")
                    else:
                        timestampLogStatusLabel.configure(text="Time in input was invalid. Please try again.")
                elif employeeStatus == 'in':
                    employeeTimeIn = ""
                    ems.addEmployeeToLog(employeeID, employeeStatus, employeeTimeIn)
                    employeeScrolledText.configure(state=NORMAL)
                    employeeScrolledText.delete('1.0', END)
                    employeeScrolledText.insert(INSERT, ems.getUpdatedLog())
                    employeeScrolledText.configure(state=DISABLED)
                    timestampLogStatusLabel.configure(text="")
                else:
                    timestampLogStatusLabel.configure(text="Status was improperly set. Please try again.")
        else:
            timestampLogStatusLabel.configure(text="Something went wrong. Please try again.")
    except:
        timestampLogStatusLabel.configure(text="Something went wrong. Please try again.")

timestampLogAddButton = Button(text="Add employee", command=addEmployeeToLog, state=DISABLED)
timestampLogAddButton.grid(row=5, column=2)

generateLogFileLabel = Label(text="Generate the timestamp log file.\nThis action will finalize the log.")
generateLogFileLabel.grid(row=10, column=2)

def generateLogFile():
    try: 
        ems.createTimestampLog()
        timestampLogStatusLabel.configure(text="Log file generated. Please check application folder.")
        timestampLogAddButton.configure(state=DISABLED)
        generateLogFileButton.configure(state=DISABLED)
    except:
        timestampLogStatusLabel.configure(text="Something went wrong. Please try again.")


generateLogFileButton = Button(text="Generate", command=generateLogFile, state=DISABLED)
generateLogFileButton.grid(row=11, column=2)

window.mainloop()